# BERT finetuning on TREC-50

## Librairy

In [ ]:
# !pip install transformers==4.8.2
# !pip install datasets==1.7.0

In [ ]:
import os
import time
import pickle

import numpy as np
import torch

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

from transformers import BertTokenizer, BertTokenizerFast
from transformers import BertForSequenceClassification, AdamW
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback
from transformers.data.data_collator import DataCollatorWithPadding

from datasets import load_dataset, Dataset, concatenate_datasets

In [ ]:
# print(torch.__version__)
# print(torch.cuda.device_count())
# print(torch.cuda.is_available())
# print(torch.cuda.get_device_name(0))

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# if torch.cuda.is_available():
#     torch.set_default_tensor_type('torch.cuda.FloatTensor')

In [ ]:
device

## Global variables

In [ ]:
BATCH_SIZE = 24
NB_EPOCHS = 4

In [ ]:
RESULTS_FILE = '~/Results/BERT_finetune/trec-50_BERT_finetune_b'+str(BATCH_SIZE)+'_results.pkl'
RESULTS_PATH = '~/Results/BERT_finetune/trec-50_b'+str(BATCH_SIZE)+'/'
CACHE_DIR = '~/Data/huggignface/'         # path of your  folder

## Dataset

In [ ]:
# download dataset

raw_datasets = load_dataset('trec', cache_dir=CACHE_DIR)

In [ ]:
# tokenize

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.rename_column('label-fine', 'labels') # 'label-fine'
tokenized_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

train_dataset = tokenized_datasets["train"].shuffle(seed=42)
train_val_datasets = train_dataset.train_test_split(train_size=0.8)

train_dataset = train_val_datasets['train']
val_dataset = train_val_datasets['test']
test_dataset = tokenized_datasets["test"].shuffle(seed=42)

In [ ]:
# get number of labels

num_labels = len(set(train_dataset['labels'].tolist()))
# num_labels = 50
num_labels

## Model

#### Model

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)
model.to(device)

#### Training

In [ ]:
training_args = TrainingArguments(
    
    # output
    output_dir=RESULTS_PATH,          
    
    # params
    num_train_epochs=NB_EPOCHS,                # nb of epochs
    per_device_train_batch_size=BATCH_SIZE,    # batch size per device during training
    per_device_eval_batch_size=BATCH_SIZE,     # cf. paper Sun et al.
    learning_rate=2e-5,                        # cf. paper Sun et al.
#     warmup_steps=500,                        # number of warmup steps for learning rate scheduler
    warmup_ratio=0.1,                          # cf. paper Sun et al.
    weight_decay=0.01,                         # strength of weight decay
    
#     # eval
    evaluation_strategy="steps",
    eval_steps=10,
#     evaluation_strategy='no', # no more evaluation, takes time
    
    # log
    logging_dir=RESULTS_PATH+'logs',  
    logging_strategy='steps',
    logging_steps=10,
    
    # save
#     save_strategy='epoch',
    # save_strategy='steps',
#     load_best_model_at_end=False
    load_best_model_at_end=True               # cf. paper Sun et al.
)

In [ ]:
def compute_metrics(p):
    
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    
    return {"val_accuracy": accuracy}

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    # compute_metrics=compute_metrics,
    # callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
)

In [ ]:
results = trainer.train()

In [ ]:
training_time = results.metrics["train_runtime"]
training_time_per_epoch = training_time / training_args.num_train_epochs
training_time_per_epoch

In [ ]:
trainer.save_model(os.path.join(RESULTS_PATH, 'best_model-0'))

## Results

In [ ]:
results_d = {}
epoch = 1

ordered_files = sorted( [f for f in os.listdir(RESULTS_PATH) 
                         if (not f.endswith("logs")) and (f.startswith("best")) # best model eval only
                        ],
                         key=lambda x: int(x.split('-')[1]) )

for filename in ordered_files:
    
    print(filename)
    
    # load model
    model_file = os.path.join(RESULTS_PATH, filename)
    finetuned_model = BertForSequenceClassification.from_pretrained(model_file, num_labels=num_labels)
    finetuned_model.to(device)
    finetuned_model.eval()

    # compute test acc
    test_trainer = Trainer(finetuned_model, data_collator=DataCollatorWithPadding(tokenizer))

    raw_preds, labels, _ = test_trainer.predict(test_dataset)
    preds = np.argmax(raw_preds, axis=1)

    test_acc = accuracy_score(y_true=labels, y_pred=preds)
    
#     results_d[filename] = (test_acc, training_time_per_epoch*epoch)
    results_d[filename] = test_acc # best model evaluation only
    
    print((test_acc, training_time_per_epoch*epoch))
    
    epoch += 1
    
results_d['training_time'] = training_time

In [ ]:
# save results

with open(RESULTS_FILE, 'wb') as fh:
    pickle.dump(results_d, fh)

In [ ]:
results_d